In [28]:
import pandas as pd
import os

=== List Raw File Name ===

In [29]:
INPUT_FILES = [
    '../raw/historical/open-meteo-2020.csv',
    '../raw/historical/open-meteo-2021.csv',
    '../raw/historical/open-meteo-2022.csv',
    '../raw/historical/open-meteo-2023-2024.csv',
    '../raw/historical/open-meteo-2025.csv'
]

OUTPUT_FOLDER = './historical'

Define feature prepare for model

In [30]:
COLUMNS_TO_KEEP = [
    'time',
    'rain',
    'precipitation',
    'temperature_2m',
    'dew_point_2m',
    'soil_temperature_0_to_7cm',
    'pressure_msl',
    'cloud_cover',
    'wind_speed_100m'
]

In [31]:
INPUT_FILE = INPUT_FILES[1]  # 2021
df_raw = pd.read_csv(INPUT_FILE)
df_raw.head()

,time,temperature_2m,rain,precipitation,dew_point_2m,snowfall,snow_depth,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,wind_gusts_10m,wind_direction_100m,wind_speed_100m,pressure_msl,cloud_cover,et0_fao_evapotranspiration
0,2021-01-01T00:00,19.4,0.0,0.0,14.5,0.0,0.0,19.0,20.9,43.6,38,35.9,1021.4,55,0.09
1,2021-01-01T01:00,19.6,0.0,0.0,13.9,0.0,0.0,19.1,20.9,42.5,37,36.2,1022.2,89,0.12
2,2021-01-01T02:00,19.8,0.0,0.0,13.7,0.0,0.0,19.4,20.8,44.6,33,36.7,1022.7,57,0.16
3,2021-01-01T03:00,20.4,0.0,0.0,13.6,0.0,0.0,19.9,20.8,44.6,31,33.8,1022.7,34,0.23
4,2021-01-01T04:00,20.3,0.0,0.0,14.0,0.0,0.0,20.6,20.8,43.2,29,33.0,1022.3,74,0.28


Checking missing value in any row

In [32]:
missing_cols = [col for col in COLUMNS_TO_KEEP if col not in df_raw.columns]
if missing_cols:
    print(f"Warining missing cols: {missing_cols}")


In [33]:
df_filtered = df_raw[COLUMNS_TO_KEEP]
df_filtered.head()

,time,rain,precipitation,temperature_2m,dew_point_2m,soil_temperature_0_to_7cm,pressure_msl,cloud_cover,wind_speed_100m
0,2021-01-01T00:00,0.0,0.0,19.4,14.5,19.0,1021.4,55,35.9
1,2021-01-01T01:00,0.0,0.0,19.6,13.9,19.1,1022.2,89,36.2
2,2021-01-01T02:00,0.0,0.0,19.8,13.7,19.4,1022.7,57,36.7
3,2021-01-01T03:00,0.0,0.0,20.4,13.6,19.9,1022.7,34,33.8
4,2021-01-01T04:00,0.0,0.0,20.3,14.0,20.6,1022.3,74,33.0


Add new columns

In [34]:
df_filtered['time'] = pd.to_datetime(df_raw['time'])

df_filtered['year'] = df_filtered['time'].dt.year
df_filtered['month'] = df_filtered['time'].dt.month
df_filtered['day'] = df_filtered['time'].dt.day
df_filtered['hour'] = df_filtered['time'].dt.hour


df_filtered.head()

C:\Users\minhp\AppData\Local\Temp\ipykernel_17604\734620765.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['time'] = pd.to_datetime(df_raw['time'])
C:\Users\minhp\AppData\Local\Temp\ipykernel_17604\734620765.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['year'] = df_filtered['time'].dt.year
C:\Users\minhp\AppData\Local\Temp\ipykernel_17604\734620765.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,time,rain,precipitation,temperature_2m,dew_point_2m,soil_temperature_0_to_7cm,pressure_msl,cloud_cover,wind_speed_100m,year,month,day,hour
0,2021-01-01 00:00:00,0.0,0.0,19.4,14.5,19.0,1021.4,55,35.9,2021,1,1,0
1,2021-01-01 01:00:00,0.0,0.0,19.6,13.9,19.1,1022.2,89,36.2,2021,1,1,1
2,2021-01-01 02:00:00,0.0,0.0,19.8,13.7,19.4,1022.7,57,36.7,2021,1,1,2
3,2021-01-01 03:00:00,0.0,0.0,20.4,13.6,19.9,1022.7,34,33.8,2021,1,1,3
4,2021-01-01 04:00:00,0.0,0.0,20.3,14.0,20.6,1022.3,74,33.0,2021,1,1,4


In [37]:
def preprocess_weather_file(input_path):
    df = pd.read_csv(input_path)

    missing_cols = [col for col in COLUMNS_TO_KEEP if col not in df.columns]
    if missing_cols:
        print(f"⚠️ Warning: Missing columns in {input_path}: {missing_cols}")

    df_filtered = df[COLUMNS_TO_KEEP].copy()

    df_filtered['time'] = pd.to_datetime(df_filtered['time'])
    df_filtered['year'] = df_filtered['time'].dt.year
    df_filtered['month'] = df_filtered['time'].dt.month
    df_filtered['day'] = df_filtered['time'].dt.day
    df_filtered['hour'] = df_filtered['time'].dt.hour
    df_filtered['date'] = df_filtered['time'].dt.date

    return df_filtered

In [38]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [39]:
for path in INPUT_FILES:
    df_processed = preprocess_weather_file(path)

    filename = os.path.basename(path).replace("open-meteo", "weather_processed")
    output_path = os.path.join(OUTPUT_FOLDER, filename)
    df_processed.to_csv(output_path, index=False)